# Custom Mesh Swarms

**Building Specialized Agent Teams with Direct Communication**

---

Welcome to this comprehensive tutorial on **Custom Mesh Swarms** in the Strands framework! This notebook teaches you how to build sophisticated swarm architectures where specialized agents communicate directly with each other. By the end of this 10-minute tutorial, you'll create a customer feedback analysis swarm with research, creative, critical, and summarizer agents working together.

### 🎯 What You'll Learn

In this tutorial, you will:
- Build mesh swarm architectures from scratch
- Create specialized agents with distinct roles
- Implement SharedMemory for agent communication
- Execute multi-phase swarm processing
- Analyze customer feedback with collective intelligence
- Compare custom swarms vs built-in swarm tool

### 🕸️ What is Mesh Swarm Architecture?

**Mesh Architecture** means every agent can communicate with every other agent:
- **Direct Communication**: Agents share insights immediately
- **Collective Knowledge**: All agents benefit from each other's discoveries
- **Specialized Roles**: Each agent has a specific expertise
- **Multi-Phase Processing**: Agents refine their work based on peer feedback

### 🏗️ Why Build Custom Swarms?

Custom swarms offer advanced capabilities:
- **Specialized System Prompts**: Tailor each agent for specific roles
- **Fine-grained Control**: Manage exactly how agents interact
- **Complex Workflows**: Multi-phase processing with feedback loops
- **Custom Memory Systems**: Implement sophisticated knowledge sharing
- **Performance Optimization**: Optimize for your specific use case

## 📦 Step 1: Installing Required Packages

### Overview
Let's install the necessary packages for building custom mesh swarms.

### 📚 Packages We'll Install
- **strands-agents**: Core framework with agent support
- **boto3**: For AWS Bedrock integration
- **threading**: For thread-safe SharedMemory

In [ ]:
# Install required packages
%pip install strands-agents boto3 -q

print("✅ All packages installed successfully!")
print("   Ready to build custom mesh swarms! 🕸️")

## 🔐 Step 2: Setting Up AWS Authentication

### Overview
We'll configure AWS Bedrock for our specialized agents.

### 🔑 Authentication Options
1. **AWS Profile** (Recommended for development)
2. **Environment Variables**
3. **Direct Credentials** (Less secure)
4. **IAM Roles** (Recommended for production)

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel
import threading
import time
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
from datetime import datetime

# Configure AWS session
session = boto3.Session(
    # aws_access_key_id='your_access_key',
    # aws_secret_access_key='your_secret_key',
    # aws_session_token='your_session_token',  # If using temporary credentials
    # region_name='us-west-2',
    profile_name='default'  # Optional: Use a specific AWS profile
)

# Create a Bedrock model instance
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

print("✅ AWS Bedrock configured successfully!")
print(f"   Model: Claude 3.7 Sonnet")
print(f"   Profile: {session.profile_name}")

## 🧠 Step 3: Building SharedMemory System

### Thread-Safe Communication Hub

Let's build a sophisticated SharedMemory system that allows agents to share knowledge safely:

In [ ]:
class SharedMemory:
    """Thread-safe shared memory system for agent communication"""
    
    def __init__(self):
        self.memory = {}
        self.lock = threading.Lock()
        self.contributions = []
        self.current_phase = 1
        
    def store(self, agent_id: str, content: str, phase: int = None):
        """Store content from an agent with metadata"""
        with self.lock:
            timestamp = datetime.now().isoformat()
            
            contribution = {
                "agent_id": agent_id,
                "content": content,
                "phase": phase or self.current_phase,
                "timestamp": timestamp
            }
            
            self.contributions.append(contribution)
            
            # Update agent's memory
            if agent_id not in self.memory:
                self.memory[agent_id] = []
            self.memory[agent_id].append(contribution)
            
    def get_all_contributions(self, exclude_agent: str = None, phase: int = None) -> List[Dict]:
        """Get all contributions, optionally excluding an agent or filtering by phase"""
        with self.lock:
            filtered = self.contributions
            
            if exclude_agent:
                filtered = [c for c in filtered if c["agent_id"] != exclude_agent]
                
            if phase:
                filtered = [c for c in filtered if c["phase"] == phase]
                
            return filtered
    
    def get_phase_summary(self, phase: int) -> str:
        """Get a summary of all contributions from a specific phase"""
        contributions = self.get_all_contributions(phase=phase)
        
        if not contributions:
            return "No contributions found for this phase."
            
        summary = f"Phase {phase} Contributions:\n"
        for contrib in contributions:
            summary += f"\n{contrib['agent_id']}: {contrib['content'][:200]}..."
            
        return summary
    
    def advance_phase(self):
        """Move to the next phase"""
        with self.lock:
            self.current_phase += 1
            
    def get_stats(self) -> Dict[str, Any]:
        """Get memory statistics"""
        with self.lock:
            agent_counts = {}
            for contrib in self.contributions:
                agent_id = contrib["agent_id"]
                agent_counts[agent_id] = agent_counts.get(agent_id, 0) + 1
                
            return {
                "total_contributions": len(self.contributions),
                "current_phase": self.current_phase,
                "agents_active": len(agent_counts),
                "contributions_per_agent": agent_counts
            }

# Test the SharedMemory system
print("🧠 SHAREDMEMORY SYSTEM TEST")
print("=" * 60)

memory = SharedMemory()

# Simulate agent contributions
memory.store("research_agent", "Found market data indicating 65% customer satisfaction")
memory.store("creative_agent", "Proposed innovative solution for top complaint")
memory.store("critical_agent", "Identified potential risks in current approach")

# Show stats
stats = memory.get_stats()
print(f"Memory Stats: {stats}")

# Show all contributions
contributions = memory.get_all_contributions()
print(f"\nContributions: {len(contributions)} total")
for contrib in contributions:
    print(f"  {contrib['agent_id']}: {contrib['content'][:50]}...")

## 👥 Step 4: Creating Specialized Agent Roles

### The Customer Feedback Analysis Team

Let's create our specialized agents, each with a distinct role in analyzing customer feedback:

In [ ]:
# Research Agent - Analyzes data and identifies patterns
research_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a Research Agent specializing in data analysis and pattern identification.
    
    Your role in the swarm:
    - Analyze customer feedback data for patterns and trends
    - Identify key issues and satisfaction metrics
    - Provide factual, data-driven insights
    - Categorize feedback themes
    
    When receiving input from other agents, evaluate if their information 
    aligns with your research findings. Focus on accuracy and supporting 
    evidence for all conclusions."""
)

# Creative Agent - Generates innovative solutions
creative_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a Creative Agent specializing in innovative problem-solving.
    
    Your role in the swarm:
    - Generate creative solutions to customer pain points
    - Think outside the box for service improvements
    - Propose innovative features and approaches
    - Build upon insights from other agents with fresh perspectives
    
    Focus on novel approaches that others might not consider. 
    Be bold with ideas while remaining practical."""
)

# Critical Agent - Evaluates and finds flaws
critical_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a Critical Agent specializing in risk analysis and quality assurance.
    
    Your role in the swarm:
    - Critically evaluate proposed solutions
    - Identify potential flaws, risks, and unintended consequences
    - Challenge assumptions made by other agents
    - Ensure recommendations are realistic and implementable
    
    Be constructive in your criticism while ensuring the final 
    solution is robust and well-thought-out."""
)

# Summarizer Agent - Synthesizes collective intelligence
summarizer_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are a Summarizer Agent specializing in synthesis and strategic recommendations.
    
    Your role in the swarm:
    - Synthesize insights from all agents into actionable recommendations
    - Create comprehensive analysis that addresses all perspectives
    - Prioritize solutions based on impact and feasibility
    - Present findings in a clear, executive-ready format
    
    Focus on creating a cohesive strategy that incorporates 
    research findings, creative solutions, and addresses critical concerns."""
)

print("👥 SPECIALIZED AGENT TEAM CREATED")
print("=" * 60)
print("🔬 Research Agent: Data analysis and pattern identification")  
print("💡 Creative Agent: Innovative solutions and fresh perspectives")
print("🔍 Critical Agent: Risk analysis and quality assurance")
print("📊 Summarizer Agent: Strategic synthesis and recommendations")

## 🔄 Step 5: Implementing Mesh Communication

### Multi-Phase Swarm Processing

Now let's implement the mesh communication pattern where agents work in phases and share insights:

In [ ]:
class MeshSwarm:
    """Custom mesh swarm implementation"""
    
    def __init__(self, agents: Dict[str, Agent], shared_memory: SharedMemory):
        self.agents = agents
        self.memory = shared_memory
        
    def execute_phase(self, phase: int, task: str, agent_ids: List[str] = None) -> Dict[str, str]:
        """Execute a swarm phase with specified agents"""
        if agent_ids is None:
            agent_ids = list(self.agents.keys())
            
        print(f"\n🔄 PHASE {phase}: Processing with {len(agent_ids)} agents")
        print("-" * 50)
        
        results = {}
        
        for agent_id in agent_ids:
            print(f"   Processing with {agent_id}...")
            
            # Get context from other agents (excluding self)
            peer_contributions = self.memory.get_all_contributions(
                exclude_agent=agent_id,
                phase=phase-1 if phase > 1 else None
            )
            
            # Build context-aware prompt
            if peer_contributions and phase > 1:
                context = "\n\nInput from other agents:\n"
                for contrib in peer_contributions[-3:]:  # Last 3 contributions
                    context += f"\n{contrib['agent_id']}: {contrib['content'][:300]}..."
                    
                full_prompt = f"{task}{context}\n\nProvide your analysis considering the above input:"
            else:
                full_prompt = task
                
            # Get agent response
            response = self.agents[agent_id](full_prompt)
            result_str = str(response)
            results[agent_id] = result_str
            
            # Store in shared memory
            self.memory.store(agent_id, result_str, phase)
            
        return results
    
    def run_complete_analysis(self, task: str) -> Dict[str, Any]:
        """Run complete multi-phase mesh swarm analysis"""
        print("🕸️ STARTING MESH SWARM ANALYSIS")
        print("=" * 60)
        
        start_time = time.time()
        
        # Phase 1: Independent analysis
        print("\n📋 Task:", task)
        phase1_results = self.execute_phase(1, task)
        
        # Advance to phase 2
        self.memory.advance_phase()
        
        # Phase 2: Collaborative refinement  
        phase2_task = f"""Based on the initial analysis, refine your perspective on: {task}
        
        Consider the insights shared by other agents and build upon or challenge their findings."""
        
        phase2_results = self.execute_phase(2, phase2_task, 
                                          ["research_agent", "creative_agent", "critical_agent"])
        
        # Advance to phase 3
        self.memory.advance_phase()
        
        # Phase 3: Final synthesis
        synthesis_task = f"""Synthesize all analysis into final recommendations for: {task}
        
        Create a comprehensive report that incorporates research findings, creative solutions, 
        and addresses critical concerns. Provide specific, actionable recommendations."""
        
        phase3_results = self.execute_phase(3, synthesis_task, ["summarizer_agent"])
        
        execution_time = time.time() - start_time
        
        return {
            "task": task,
            "execution_time": execution_time,
            "phase1_results": phase1_results,
            "phase2_results": phase2_results,
            "final_synthesis": phase3_results["summarizer_agent"],
            "memory_stats": self.memory.get_stats()
        }

# Create the mesh swarm
agents_dict = {
    "research_agent": research_agent,
    "creative_agent": creative_agent,  
    "critical_agent": critical_agent,
    "summarizer_agent": summarizer_agent
}

memory = SharedMemory()
mesh_swarm = MeshSwarm(agents_dict, memory)

print("🕸️ MESH SWARM SYSTEM READY")
print("   4 specialized agents with shared memory communication")

## 📊 Step 6: Customer Feedback Analysis

### Real-World Application

Let's test our mesh swarm with a realistic customer service scenario:

In [ ]:
# Customer feedback analysis task
customer_feedback_task = """Analyze the following customer feedback data for our e-commerce platform:

RECENT CUSTOMER FEEDBACK:
- "Delivery was 3 days late, no communication about delay" (Rating: 2/5)
- "Great product quality but checkout process is confusing" (Rating: 3/5)  
- "Love the mobile app design, very intuitive!" (Rating: 5/5)
- "Customer service took 48 hours to respond to my question" (Rating: 2/5)
- "Product arrived damaged, return process was smooth though" (Rating: 3/5)
- "Website loads slowly on my computer" (Rating: 2/5)
- "Best prices I've found anywhere, will order again" (Rating: 4/5)
- "Search function doesn't work well, hard to find products" (Rating: 2/5)

TASK: Provide comprehensive analysis with specific recommendations for improvement."""

# Run the mesh swarm analysis
feedback_analysis = mesh_swarm.run_complete_analysis(customer_feedback_task)

print("\n📊 CUSTOMER FEEDBACK ANALYSIS RESULTS")
print("=" * 60)
print(f"⏱️  Total execution time: {feedback_analysis['execution_time']:.2f} seconds")
print(f"🧠 Memory stats: {feedback_analysis['memory_stats']}")

print("\n📋 FINAL SYNTHESIS:")
print("-" * 40)
print(feedback_analysis["final_synthesis"])

## ⚡ Step 7: Performance Comparison

### Custom Mesh vs Built-in Swarm Tool

Let's compare our custom mesh swarm with the built-in swarm tool:

In [ ]:
# Import the built-in swarm tool for comparison
from strands_tools import swarm

# Create agent with built-in swarm tool
builtin_swarm_agent = Agent(
    model=bedrock_model,
    tools=[swarm],
    system_prompt="You coordinate swarms to analyze customer feedback and provide recommendations."
)

# Test the same task with built-in swarm
print("⚡ PERFORMANCE COMPARISON: Custom Mesh vs Built-in Swarm")
print("=" * 60)

comparison_task = """Analyze customer complaints about slow website performance 
and confusing checkout process. Provide specific technical and UX recommendations."""

# Built-in swarm test
print("\n🔧 Built-in Swarm Tool:")
builtin_start = time.time()

builtin_result = builtin_swarm_agent.tool.swarm(
    task=comparison_task,
    swarm_size=4,
    coordination_pattern="collaborative"
)

builtin_time = time.time() - builtin_start
builtin_length = len(builtin_result["content"])

print(f"   ⏱️  Time: {builtin_time:.2f} seconds")
print(f"   📊 Response length: {builtin_length} characters")
print(f"   📝 Preview: {builtin_result['content'][:200]}...")

# Custom mesh swarm test
print("\n🕸️ Custom Mesh Swarm:")
custom_start = time.time()

# Reset memory for fair comparison
memory = SharedMemory()
mesh_swarm = MeshSwarm(agents_dict, memory)

custom_result = mesh_swarm.run_complete_analysis(comparison_task)
custom_time = custom_result['execution_time']
custom_length = len(custom_result['final_synthesis'])

print(f"   ⏱️  Time: {custom_time:.2f} seconds")
print(f"   📊 Response length: {custom_length} characters")
print(f"   📝 Preview: {custom_result['final_synthesis'][:200]}...")

# Comparison analysis
print("\n📈 COMPARISON ANALYSIS:")
print("=" * 60)
print(f"Built-in Swarm: {builtin_time:.2f}s, {builtin_length} chars")
print(f"Custom Mesh: {custom_time:.2f}s, {custom_length} chars")
print(f"Speed difference: {abs(custom_time - builtin_time):.2f}s")
print(f"Detail difference: {custom_length / builtin_length:.1f}x")

# Show advantages of each approach
print("\n💡 APPROACH ADVANTAGES:")
print("-" * 40)
print("Built-in Swarm Tool:")
print("  ✅ Quick to implement")
print("  ✅ Less code required") 
print("  ✅ Automatic coordination")

print("\nCustom Mesh Swarm:")
print("  ✅ Specialized agent roles")
print("  ✅ Multi-phase processing")
print("  ✅ Detailed memory tracking")
print("  ✅ Full control over communication")

## 🏗️ Step 8: Advanced Mesh Patterns

### Extending Your Swarm Architecture

Let's explore advanced patterns for custom mesh swarms:

In [ ]:
# Advanced pattern: Conditional agent participation
class ConditionalMeshSwarm(MeshSwarm):
    """Enhanced mesh swarm with conditional agent participation"""
    
    def __init__(self, agents: Dict[str, Agent], shared_memory: SharedMemory):
        super().__init__(agents, shared_memory)
        self.agent_specialties = {
            "research_agent": ["data", "analysis", "metrics", "statistics"],
            "creative_agent": ["innovation", "design", "features", "solutions"],
            "critical_agent": ["risk", "problems", "issues", "concerns"],
            "summarizer_agent": ["summary", "recommendations", "strategy"]
        }
    
    def select_relevant_agents(self, task: str, phase: int) -> List[str]:
        """Select agents based on task content and phase"""
        task_lower = task.lower()
        
        if phase == 1:
            # Phase 1: All agents participate
            return list(self.agents.keys())
        elif phase == 2:
            # Phase 2: Select based on task keywords
            relevant_agents = []
            for agent_id, keywords in self.agent_specialties.items():
                if any(keyword in task_lower for keyword in keywords):
                    relevant_agents.append(agent_id)
            return relevant_agents or ["research_agent", "creative_agent"]
        else:
            # Phase 3: Always use summarizer
            return ["summarizer_agent"]
    
    def run_adaptive_analysis(self, task: str) -> Dict[str, Any]:
        """Run analysis with adaptive agent selection"""
        print("🧠 ADAPTIVE MESH SWARM ANALYSIS")
        print("=" * 60)
        
        start_time = time.time()
        
        # Phase 1: Initial analysis with all agents
        selected_agents = self.select_relevant_agents(task, 1)
        print(f"\nPhase 1 agents: {selected_agents}")
        phase1_results = self.execute_phase(1, task, selected_agents)
        self.memory.advance_phase()
        
        # Phase 2: Targeted refinement
        selected_agents = self.select_relevant_agents(task, 2)
        print(f"Phase 2 agents: {selected_agents}")
        phase2_task = f"Refine analysis for: {task}"
        phase2_results = self.execute_phase(2, phase2_task, selected_agents)
        self.memory.advance_phase()
        
        # Phase 3: Synthesis
        selected_agents = self.select_relevant_agents(task, 3)
        print(f"Phase 3 agents: {selected_agents}")
        phase3_task = f"Create final recommendations for: {task}"
        phase3_results = self.execute_phase(3, phase3_task, selected_agents)
        
        execution_time = time.time() - start_time
        
        return {
            "task": task,
            "execution_time": execution_time,
            "adaptive_selection": True,
            "final_result": phase3_results["summarizer_agent"],
            "memory_stats": self.memory.get_stats()
        }

# Test adaptive swarm
print("🧠 TESTING ADAPTIVE AGENT SELECTION")
print("=" * 60)

memory = SharedMemory()
adaptive_swarm = ConditionalMeshSwarm(agents_dict, memory)

innovation_task = """Our mobile app needs innovative features to increase user engagement. 
Current retention rate is 30% after first week. Design creative solutions."""

adaptive_result = adaptive_swarm.run_adaptive_analysis(innovation_task)

print(f"\n📊 Adaptive Analysis Results:")
print(f"⏱️  Execution time: {adaptive_result['execution_time']:.2f} seconds")
print(f"🎯 Final recommendations: {adaptive_result['final_result'][:300]}...")

## 🎉 Congratulations!

### 🏆 What You've Accomplished

In this tutorial, you've successfully:
- ✅ Built custom mesh swarm architectures from scratch
- ✅ Created specialized agents with distinct roles and expertise
- ✅ Implemented thread-safe SharedMemory for agent communication
- ✅ Executed sophisticated multi-phase swarm processing
- ✅ Analyzed real customer feedback with collective intelligence
- ✅ Compared custom vs built-in swarm approaches
- ✅ Extended swarms with adaptive agent selection

### 🕸️ The Power of Custom Mesh Swarms

You now have the skills to:
- **Build specialized agent teams** with precise roles and expertise
- **Implement sophisticated communication** between agents
- **Create multi-phase workflows** with iterative refinement
- **Optimize swarm performance** for specific use cases
- **Scale complexity** beyond what built-in tools provide

### 💡 Key Takeaways

1. **Specialization**: Dedicated agents outperform generalists in complex tasks
2. **Communication**: SharedMemory enables sophisticated knowledge sharing
3. **Multi-Phase Processing**: Iterative refinement improves final quality
4. **Mesh Architecture**: Direct agent communication enables richer collaboration
5. **Customization**: Full control allows optimization for specific domains

### 🚀 When to Use Custom vs Built-in Swarms

**Use Built-in Swarm Tool for:**
- Quick prototypes and simple tasks
- General-purpose analysis
- When development speed matters
- Standard coordination patterns

**Use Custom Mesh Swarms for:**
- Complex, domain-specific analysis
- Multi-phase processing requirements
- Specialized agent roles
- Advanced communication patterns
- Production systems requiring fine-tuned control

### 📚 Resources

- [Strands Swarm Documentation](https://strandsagents.com/0.1.x/user-guide/concepts/multi-agent/swarm/)
- [Multi-Agent Communication Patterns](https://strandsagents.com/0.1.x/user-guide/concepts/multi-agent/)
- [Agent Architecture Guide](https://strandsagents.com/0.1.x/user-guide/concepts/agents/)

### 💪 Your Challenge

Build a custom mesh swarm for your domain:
```python
# Your challenge: Create domain-specific agents
domain_agents = {
    "specialist_1": Agent(model=bedrock_model, system_prompt="Your expertise..."),
    "specialist_2": Agent(model=bedrock_model, system_prompt="Your expertise..."),
    "specialist_3": Agent(model=bedrock_model, system_prompt="Your expertise..."),
    "synthesizer": Agent(model=bedrock_model, system_prompt="Your synthesis role...")
}

# Build and test your swarm
memory = SharedMemory()
your_swarm = MeshSwarm(domain_agents, memory)
result = your_swarm.run_complete_analysis("Your complex domain task...")
```

### 🌟 What's Next?

Ready to explore:
- **Production Swarm Systems**: Enterprise deployment patterns
- **Advanced Coordination**: Hierarchical and hybrid architectures  
- **Performance Optimization**: Scaling and efficiency techniques
- **Integration Patterns**: Combining swarms with workflows and RAG
- **Monitoring & Observability**: Production swarm management

### 🎊 Final Thoughts

Custom mesh swarms represent the cutting edge of multi-agent AI systems. By giving agents specialized roles and enabling rich communication, you can tackle complex problems that require diverse expertise and collaborative intelligence.

The mesh is where individual intelligence becomes collective genius!

Happy swarming! 🕸️🤖✨